In [615]:
from collections import Counter

In [616]:
with open('day7.txt') as f:
    lines = [line for line in f]   

In [617]:
part_two = True

In [618]:
if part_two:
    rankings = {"A":14,"K":13,"Q":12,"J":0,
                "T":11,"9":10,"8":9,"7":8,
                "6":7, "5":6, "4":5,"3":4,
                "2":3}
else:
    rankings = {"A":14,"K":13,"Q":12,"J":11,
            "T":10,"9":9,"8":8,"7":7,
            "6":6, "5":5, "4":4,"3":3,
            "2":2}

In [619]:
# If the card contains a jack, we can use tricks to improve the hand, here are the rules:

def find_jack_based_hand(hand):
    current_score = get_score(hand)
    counter = Counter(hand)
    counts = list(counter.values())
    
    #can always use trick to get five of a kind if the original score is one of these
    if current_score in ["five_of_a_kind","four_of_a_kind","full_house"]:
        return "AAAAA"
    if current_score == "three_of_a_kind":
        if counter["J"]==3:
            return hand #no improvement
        else: 
            #there must be only one J
            return "JJJJQ" #some arbitrary four of a kind is the best we can do
    if current_score == "two_pair":
        if counter["J"]==2:
            return "JJJJQ" #some arbitrary four of a kind
        if counter["J"]==1:
            return "JJJQQ" #full house is the best we can do
    if current_score == "one_pair":
            return "JJJAB" #arbitrary three of a kind
    if current_score == "high_card":
        return "JJK23" #best we can do is a pair
        

        
    

In [620]:
# Compute score based on the hand, with option of improving things by fiddling with jacks if doing part 2

def get_score(hand):

    counter = Counter(hand)
    counts = list(counter.values())
    counts.sort()


    score = "high_card"
    
    #Check if five of a kind:
    if all([i==hand[0] for i in hand]):
        score = "five_of_a_kind"
    
    #Check if four of a kind
    elif counts == [1,4]:
        score = "four_of_a_kind"
 
    #Check if full house
    elif counts == [2,3]:
        score = "full_house"
    
    elif counts == [1,1,3]:
        score = "three_of_a_kind"
    #Check if two pair
    elif counts == [1,2,2]:
        score = "two_pair"
        
    #Check if one pair
    elif counts == [1,1,1,2]:
        score = "one_pair"
        
    else:
        score = "high_card"
    
    return score    


def determine_hand(hand):
    # Find if five/four/three of a kind, full house, two pair, one pair or high card
    
    if part_two:
        if "J" in hand:
            hand = find_jack_based_hand(hand)
            
    return get_score(hand)
    

In [621]:
# Find which card is higher ranked if they are of the same type.
def distinguish_two_similar_hands(card_1,card_2):
    for i in range(6):
        if abs(rankings[card_1[i]]-rankings[card_2[i]])!=0:
            if rankings[card_1[i]]>rankings[card_2[i]]:
                return 0
            else:
                return 1
        
    
mapping = {"five_of_a_kind":6,"four_of_a_kind":5,"full_house":4,"three_of_a_kind":3,"two_pair":2,"one_pair":1,"high_card":0}


def determine_ranking(score_1,score_2):
    if mapping[score_1] > mapping[score_2]:
        return 0
    else:
        return 1

# Find which card is higher ranked.
def compare_two_cards(card_1,card_2):
    score_one = determine_hand(card_1)
    score_two = determine_hand(card_2)
    
    # if they are the same type of hand
    if score_one == score_two:
        return distinguish_two_similar_hands(card_1,card_2)
    
    else:
        return determine_ranking(score_one,score_two)
        
        
        
        
        

In [622]:
# Build dataset of hands, bids, and the type of hand
dataset = []
for line in lines:
    hand,bid=line.split(" ")
    bid = int(bid)
    score = determine_hand(hand)
    dataset.append((hand,score,mapping[score],bid))
    

In [623]:
dataset.sort(key = lambda x: x[2],reverse=True) #Sort based on score

In [624]:
def determine_rankings_amongst_same_scores(list_of_hands):
    final_list = [list_of_hands[0]]
    for hand in list_of_hands[1:]:
        if distinguish_two_similar_hands(hand[0],final_list[0][0]) == 0:
            final_list = [hand]+final_list
        elif len(final_list)==1:
            final_list = final_list +[hand]
        else:
            j=0
            while j<=len(final_list)-1 and distinguish_two_similar_hands(hand[0],final_list[j][0])!=0:
                j+=1
            final_list = final_list[:j] + [hand] + final_list[j:]
    return final_list

In [625]:
# Sort by the type of hand
collections = {6:[],5:[],4:[],3:[],2:[],1:[],0:[]}
for data in dataset:
    collections[data[2]].append(data)
    

In [626]:
# Amongst each type of hand, update rankings based on how individual hands compare
for i in range(7):
    if len(collections[i])!=0:
        collections[i] = determine_rankings_amongst_same_scores(collections[i])


In [627]:
ranking = collections.values()
ranking = [r for r in ranking if len(r)!=0]


In [628]:
ranking = [item for sublist in ranking for item in sublist]


In [629]:
ranking.reverse()

In [630]:
score = 0
for i in range(len(ranking)):
    score+= (i+1)*ranking[i][3]
score
    

250057090